In [1]:
from skimage import io
import dlib
import numpy as np

import os
import re
import skimage.transform as tr
from PIL import Image,ImageDraw

# Подготовка

### Считывание массива ключевых точек лица

In [2]:
face_template=np.load('face_template.npy')

### Считывание фотографии себя

In [3]:
#base_dir = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
#path_me=os.path.join(base_dir, "face_net_mkr\me_and_dots")
path_me="../face_net_mkr/me_and_dots/me.jpg"

image=io.imread(path_me)
detector=dlib.get_frontal_face_detector()
face_rects=list(detector(image,1))
face_rect=face_rects[0]
print(face_rect)


[(291, 242) (513, 464)]


### Считываем обученную модель предиктора для сегментации

In [4]:
dlib_predictor_path="../face_net_mkr/shape_predictor_68_face_landmarks.dat"
predictor = dlib.shape_predictor(dlib_predictor_path)
points = predictor(image, face_rect)
landmarks = np.array(list(map(lambda p: [p.x, p.y], points.parts())))

### Создание все точек контура лица на изображении

In [5]:
photo=Image.open(path_me)
pix=photo.load()
draw=ImageDraw.Draw(photo)
N=2

for point in landmarks:

    draw.ellipse(((point[0]-N,point[1]-N),(point[0]+N,point[1]+N)),fill=(0,139,139))
photo.save('res4.jpg','JPEG')


### Выбор главных 3х точек

In [6]:
INNER_EYES_AND_BOTTOM_LIP = [39, 42, 57]
proper_landmarks = 227 * face_template[INNER_EYES_AND_BOTTOM_LIP]
current_landmarks = landmarks[INNER_EYES_AND_BOTTOM_LIP]

### Создание 3 главных точек

In [7]:
photo=Image.open(path_me)
pix=photo.load()
draw=ImageDraw.Draw(photo)
for point in current_landmarks:

    draw.ellipse(((point[0]-N,point[1]-N),(point[0]+N,point[1]+N)),fill=(255,0,0))
photo.save('res20.jpg','JPEG')


## Создадим функцию, в которой совмещены все вышеперечисленные методы обработки и также обрезка фотографий с помощью аффиного отображения


In [8]:

def make_photo(path):

    files_names=os.listdir(path)
    folder_names=list(set([ name[:4] for name in files_names]))

    os.mkdir('dataset')
    for i,name in enumerate(folder_names):
        os.mkdir('dataset/'+str(i))
    size=128
        
    face_template=np.load('face_template.npy')
    predictor=dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
    detector=dlib.get_frontal_face_detector()
    INNER_EYES_AND_BOTTOM_LIP = [39, 42, 57]
    
    for name in files_names:
      image=io.imread(path+'/'+name)
      
      face_rects=list(detector(image,1))
      try:
        face_rect=face_rects[0]
    
        points=predictor(image,face_rect)
        landmarks=np.array(list(map(lambda p: [p.x,p.y],points.parts())))
    
        proper_landmarks=size*face_template[INNER_EYES_AND_BOTTOM_LIP]
        current_landmarks = landmarks[INNER_EYES_AND_BOTTOM_LIP]
    
        A = np.hstack([current_landmarks, np.ones((3, 1))]).astype(np.float64)
        B = np.hstack([proper_landmarks, np.ones((3, 1))]).astype(np.float64)
        T = np.linalg.solve(A, B).T
    
        wrapped = tr.warp(
        image,
        tr.AffineTransform(T).inverse,
        output_shape=(size, size),
        mode='constant',
        cval=0,
        preserve_range=True
        )
        im = Image.fromarray(wrapped.astype(np.uint8))
        way=name[:4]

        namel=name[6:10]
        im.save('dataset/'+str(folder_names.index(way))+'/'+str(namel)+'.jpg')
      except Exception:
        pass

In [9]:
make_photo('photo')   
  

FileExistsError: [WinError 183] Невозможно создать файл, так как он уже существует: 'dataset'

## Удалим тех людей в датасете, чье количество фотографий меньше порогового значения

In [ ]:
path='dataset'
N=11
files_names=os.listdir(path)
folder_names=list(set([ name[:4] for name in files_names]))
for i in folder_names:
    if len(os.listdir(path+'/'+str(i)))<N:
        for j in os.listdir(path+'/'+str(i)):
            os.remove(path+'/'+str(i)+'/'+j)
        os.rmdir(path+'/'+str(i))
